<a href="https://colab.research.google.com/github/PadmajaVB/Sexist-Statement-Detection/blob/main/gettweet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get tweets labeled with benevolent sexism using twitter API

In [9]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


**Enter twitter developer keys which will be used to comminicate to twitter API using tweepy**

In [10]:
import tweepy

# Twitter Developer keys here
# USED TO COMMUNICATE TO THE TWITTER API
consumer_key = 'DYblLECW6ZxgdNmZbOL5Vsbty'
consumer_key_secret = '5bjrh8S8rNXEVQSMDUl9DX3yYtqXSFFJWJ8r7mwDFgs0cUSF6P'
access_token = '1089903230992056320-rpusTvBh7re56F64sj7YoTCeMLrDcb'
access_token_secret = '1ksc9eaJer3c5cQc6HYVG5UCqkPdL8I3Eb1Oz1dQxieh9'


In [11]:
#set up the API for fetching
auth = tweepy.OAuthHandler(consumer_key, consumer_key_secret) 
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [16]:
# This is corpus dataset
import pandas as pd
#tweet id form (check this tsv for format)
corpusFile = "/content/drive/MyDrive/AI-3 Project/data/NLP_CSS_2017-master/benevolent_sexist.tsv"
df= pd.read_csv(corpusFile, header= None)


**Custom function to fetch tweets and store them as a list**

In [22]:
# This method creates the training set
def createTrainingSet(corpusFile):
    import csv
    import time

    c=0
    #print(corpusFile.head())
    #Get each id as part of the list
    corpus = list(df[0])

    sleepTime = 2
    trainingDataSet = []
    #Do not send more than 1500 ids at once else you will have to wait for 1 hour to cooldown
    for tweet in corpus[0:200]:
        try:
            tweetFetched = api.get_status(tweet) #get the metadeta
            print(c,"Tweet fetched = " + tweetFetched.text)
            c+=1 #counter for seeing which tweet index is being fetched
            time.sleep(sleepTime) #for the API to cool down
            trainingDataSet.append(tweetFetched.text)
        except Exception as e:
            print(c,"Inside the exception - no:2")
            print(e)
            c+=1
            print()
            time.sleep(1)
            continue
    #return in list       
    return trainingDataSet
    


In [23]:
resultFile = createTrainingSet(df)

0 Tweet fetched = RT @GemmaAnneStyles: Happy #womensday to all my sassy women, classy women, nasty women 💋👧🏾💖👩🏼‍🔬👄👵🏼 my fine women, alive women, gonna fight…
1 Inside the exception - no:2
[{'code': 144, 'message': 'No status found with that ID.'}]

2 Inside the exception - no:2
[{'code': 144, 'message': 'No status found with that ID.'}]

3 Tweet fetched = RT @girlsreallyrule: The lights are out at the Statue of Liberty...that's ONE MORE woman America CANNOT do without. #adaywithoutwomen #Ligh…
4 Inside the exception - no:2
[{'code': 63, 'message': 'User has been suspended.'}]

5 Inside the exception - no:2
[{'code': 144, 'message': 'No status found with that ID.'}]

6 Inside the exception - no:2
[{'code': 63, 'message': 'User has been suspended.'}]

7 Inside the exception - no:2
[{'code': 63, 'message': 'User has been suspended.'}]

8 Inside the exception - no:2
[{'code': 144, 'message': 'No status found with that ID.'}]

9 Inside the exception - no:2
[{'code': 144, 'message': 'No stat

**Convert to dataframe and save**

In [24]:
res= pd.DataFrame(resultFile)

In [25]:
res.to_csv('first200.csv')

In [27]:
print(res.loc[3][0])

"she parties like a man but loves like a woman"


# Preprocess tweets

**Import required libraries and datasets**

In [ ]:
import pandas as pd
import re
!pip install emoji #to remove emojis from the tweets
import emoji
csv1= pd.read_csv('first200.csv', header= None)
csv2= pd.read_csv('1500to4520.csv', header= None)
csv3= pd.read_csv('4520.csv', header= None) 

In [ ]:
csv= pd.concat([csv1, csv2, csv3]) #concatenating the files obtained by running for 3 attempts

In [ ]:
csv.drop([0], axis=1, inplace= True) #do .head() before to check if 0th col or row has to be removed
csv.drop([0], axis=0, inplace= True)

In [ ]:
csv.head()

,1
1,RT @GemmaAnneStyles: Happy #womensday to all m...
2,RT @girlsreallyrule: The lights are out at the...
3,RT @antonioguterres: We can’t achieve any of o...
4,"""she parties like a man but loves like a woman"""
5,RT @GemmaAnneStyles: Happy #womensday to all m...


**Remove retweets**

In [ ]:
csv.drop_duplicates(inplace=True) #duplicate tweets

In [ ]:
print(len(csv))
csv.head()

401


,1
1,RT @GemmaAnneStyles: Happy #womensday to all m...
2,RT @girlsreallyrule: The lights are out at the...
3,RT @antonioguterres: We can’t achieve any of o...
4,"""she parties like a man but loves like a woman"""
6,RT @xanria_018: Lucky is the man who is the fi...


In [ ]:
csv.reset_index(inplace=True)
csv.drop(['index'], axis=1,inplace= True) 

**Remove @mentions along with emojis and website mentions as they are not important for our task**

In [ ]:
def preprocess(df):
  df[1]= [re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(http?://[^\s]+))', '', ln) for ln in df[1]] #remove websites
  df[1]= [re.sub(r'http\S+', '', ln) for ln in df[1]] 
  df[1]= [re.sub('@[^\s]+', '', ln) for ln in df[1]] #remove account mentions
  df[1]= [emoji.get_emoji_regexp().sub(u'', ln) for ln in df[1]] #remove emojis
  df[1]= [ln.split() for ln in df[1]] 
  
  for i,ln in enumerate(df[1]):
    try:
      ln.remove('RT') #remove retweets
    except:
      continue
    df.iloc[i,0]=ln
  [print(ln) for ln in df[1]]
  return df

**Remove any null values**

In [ ]:
csv.dropna(inplace= True) #remove any null values

In [ ]:
csv= preprocess(csv)

In [ ]:
csv.head()

,1
0,"[Happy, #womensday, to, all, my, sassy, women,..."
1,"[The, lights, are, out, at, the, Statue, of, L..."
2,"[We, can’t, achieve, any, of, our, goals, with..."
3,"[""she, parties, like, a, man, but, loves, like..."
4,"[Lucky, is, the, man, who, is, the, first, lov..."


**Rejoin the split strings as they will be tokenized later as per the model requirements**

In [ ]:
csv=csv[1].str.join(" ")

0      Happy #womensday to all my sassy women, classy...
1      The lights are out at the Statue of Liberty......
2      We can’t achieve any of our goals without the ...
3        "she parties like a man but loves like a woman"
4      Lucky is the man who is the first love of a wo...
                             ...                        
396    Literally can eat a whole pizza to myself .......
397    'THE BIGGEST COWARD OF A MAN IS TO AWAKEN THE ...
398    We want to construct a different world of work...
399    Do not awaken the love of a woman, without the...
400    Nothing better in this world than a man making...
Name: 1, Length: 400, dtype: object

**Save the dataset**

In [ ]:
csv.to_csv("Benevolent_sexism.csv")